In [2]:
import numpy as np
import pandas as pd

from skrough.dataprep import prepare_factorized_array, prepare_factorized_vector

In [3]:
# df_train = pd.read_csv("../../1m_sdiv_balanced.csv", index_col=0)
# target_train = pd.read_csv("../../1m_sdiv_y.csv", index_col=0, header=None).reset_index(drop=True)[1]

df_train = pd.read_csv(
    "../../26m_sdiv_biased.csv", index_col=0, dtype=np.int8, nrows=1000000
)
target_train = pd.read_csv(
    "../../26m_sdiv_y.csv", index_col=0, header=None, dtype=np.int8, nrows=1000000
).reset_index(drop=True)[1]

In [4]:
x, x_counts = prepare_factorized_array(df_train.to_numpy())
y, y_count = prepare_factorized_vector(target_train.to_numpy())

# x, x_counts = prepare_factorized_array(df.to_numpy())
# y, y_count = prepare_factorized_vector(target)

In [9]:
import numpy as np

from skrough.algorithms import hooks
from skrough.algorithms.hooks.names import (
    CONFIG_CANDIDATES_MAX_COUNT,
    CONFIG_CHAOS_FUN,
    CONFIG_CONSECUTIVE_EMPTY_ITERATIONS_MAX_COUNT,
    CONFIG_DAAR_ALLOWED_RANDOMNESS,
    CONFIG_DAAR_N_OF_PROBES,
    CONFIG_EPSILON,
    CONFIG_SELECT_ATTRS_CHAOS_SCORE_BASED_MAX_COUNT,
    INPUT_X,
    INPUT_Y,
    VALUES_CHAOS_SCORE_APPROX_THRESHOLD,
    VALUES_CHAOS_SCORE_BASE,
    VALUES_CHAOS_SCORE_TOTAL,
    VALUES_GROUP_INDEX,
    VALUES_RESULT_ATTRS,
    VALUES_RESULT_OBJS,
    VALUES_X,
    VALUES_X_COUNTS,
    VALUES_Y,
    VALUES_Y_COUNT,
)
from skrough.algorithms.meta import processing, stage
from skrough.chaos_measures import entropy, gini_impurity
from skrough.chaos_score import get_chaos_score_stats
from skrough.structs.state import ProcessingState

grow_stage = stage.Stage.from_hooks(
    stop_hooks=[
        hooks.stop_hooks.stop_hook_approx_threshold,
        # hooks.stop_hooks.stop_hook_empty_iterations,
    ],
    init_hooks=None,
    pre_candidates_hooks=[
        hooks.pre_candidates_hooks.pre_candidates_hook_remaining_attrs,
    ],
    # candidates_hooks=hooks.common.process_elements.process_elements_hook_pass_everything,
    candidates_hooks=hooks.common.process_elements.create_process_elements_hook_random_choice(
        CONFIG_CANDIDATES_MAX_COUNT
    ),
    select_hooks=[
        hooks.select_hooks.select_hook_attrs_chaos_score_based,
    ],
    # filter_hooks=hooks.filter_hooks.filter_hook_attrs_first_daar,
    filter_hooks=None,
    # inner_init_hooks=hooks.inner_init_hooks.inner_init_hook_consecutive_empty_iterations_count,
    inner_init_hooks=None,
    inner_stop_hooks=hooks.inner_stop_hooks.inner_stop_hook_empty,
    inner_process_hooks=hooks.inner_process_hooks.inner_process_hook_add_first_attr,
    finalize_hooks=None,
)


msfun = processing.ProcessingMultiStage.from_hooks(
    init_multi_stage_hooks=[
        hooks.init_hooks.init_hook_factorize_data_x_y,
        hooks.init_hooks.init_hook_approx_threshold,
        hooks.init_hooks.init_hook_result_attrs_empty,
        hooks.init_hooks.init_hook_single_group_index,
    ],
    stages=[grow_stage],
    finalize_hooks=None,
    prepare_result_fun=hooks.prepare_result_hooks.prepare_result_hook_attrs_subset,
)


def compute():
    return msfun(
        input_data={
            INPUT_X: df_train.to_numpy(),
            INPUT_Y: target_train.to_numpy(),
        },
        config={
            CONFIG_CHAOS_FUN: entropy,
            CONFIG_EPSILON: 0.00,
            CONFIG_SELECT_ATTRS_CHAOS_SCORE_BASED_MAX_COUNT: 1,
            CONFIG_CANDIDATES_MAX_COUNT: 50,
            # CONFIG_DAAR_ALLOWED_RANDOMNESS: 0.2,
            # CONFIG_DAAR_N_OF_PROBES: 100,
            # CONFIG_CONSECUTIVE_EMPTY_ITERATIONS_MAX_COUNT: 1,
        },
    )


res = compute()

In [47]:
# import joblib
# result = joblib.Parallel(n_jobs=4)(
#             joblib.delayed(compute)()
#             for _ in range(10)
#         )
# result

In [12]:
from skrough.checks import check_if_approx_reduct

In [20]:
check_if_approx_reduct(
    x, x_counts, y, y_count, chaos_fun=entropy, epsilon=0.5, attrs=[68]
)

True

In [10]:
res

AttrsSubset(attrs=[68, 69, 66, 58, 59, 57])